In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
from gensim.models import KeyedVectors

In [29]:
w2v=KeyedVectors.load("/content/drive/MyDrive/FYP/CAA_NRC/Word2Vec/w2c")

In [30]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=pd.get_dummies(df["label"])
df

,label,new_text
0,0,muslim immigrant says that we will conquer ger...
1,1,my message to indian muslims kashmir is the sp...
2,2,normally never i abuse anyone on twitter but t...
3,1,heard someone suggesting holding anti caa nrc ...
4,2,the apex court has issued a notice to the unio...
...,...,...
3096,2,save hindus infiltrators anti national go to p...
3097,2,breaking bhimarmychief chandrashekhar azad has...
3098,0,tamil nadu cm edappadi k palaniswamy says citi...
3099,2,some more photos of protest meetings against c...


In [31]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [32]:
import tensorflow as tf
maxlen=maxi
max_words=len(w2v.wv.vocab.keys())
print(max_words)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

10123


In [33]:
len(w2v["happy"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


300

In [34]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  
  if word in w2v:
    embed_vector=w2v[word]
    # print(i)
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [35]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.56359738,  0.01164975,  0.27649218, ...,  0.62336755,
         0.31262058,  0.24197567],
       [ 0.72490132, -0.07310981,  0.19244824, ...,  0.54099172,
         0.7085728 ,  0.35532942],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [37]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,LSTM,Dense
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=maxi,embeddings_initializer=Constant(embed_matrix)))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=False)) 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(3,activation='softmax'))

In [38]:
import time
start = time.time()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/10
37/37 [==============================] - 3s 15ms/step - loss: 1.0996 - accuracy: 0.3656
Epoch 2/10
37/37 [==============================] - 0s 13ms/step - loss: 1.0666 - accuracy: 0.4357
Epoch 3/10
37/37 [==============================] - 0s 13ms/step - loss: 1.0466 - accuracy: 0.4409
Epoch 4/10
37/37 [==============================] - 0s 12ms/step - loss: 1.0028 - accuracy: 0.4761
Epoch 5/10
37/37 [==============================] - 0s 12ms/step - loss: 0.9804 - accuracy: 0.4903
Epoch 6/10
37/37 [==============================] - 0s 13ms/step - loss: 0.9455 - accuracy: 0.5049
Epoch 7/10
37/37 [==============================] - 0s 13ms/step - loss: 0.8779 - accuracy: 0.5312
Epoch 8/10
37/37 [==============================] - 0s 13ms/step - loss: 0.7478 - accuracy: 0.5751
Epoch 9/10
37/37 [==============================] - 0s 13ms/step - loss: 0.6399 - accuracy: 0.6959
Epoch 10/10
37/37 [==============================] - 0s 13ms/step - loss: 0.5448 - accuracy: 0.7948
Training 

In [39]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))


In [40]:
import numpy as np
y_pred=model.predict(X_test)
y_test=y_test.to_numpy()
l=[np.argmax(i) for i in y_pred]
M=[np.argmax(i) for i in y_test]
display_metrics(M,l)

0.6713917525773195
0.6472742733819888
0.7067465653180477
0.6348213413195508
